适配器模式      

适配器是一个有助于我们把两个不适配的结构变得可适配的结构化设计模式。首先，让我们来回答不适配借口的真实含义。如果我们拥有一个旧组件，而且我们想要把它用于一个新系统，或者是我们想要在一个旧系统中使用新组建，那么这两种方式下，很少有不要求不做任何代码改变的情况。但是改变代码也不少总是可行的，除非是因为我们不想访问他（例如，组建作为一个扩展库而被提供），或者是因为根本不可行。
电商系统是一个已知的例子。假设我们使用一个包含了calculate_total(order)函数的电商系统。该函数封装了一个订单的总价，但是只用于丹麦克朗（DKK）。我们的客户要求我们添加对更多流行货币对支持是合理的，比如美国美元（USD），和欧元（EUR）。如果我们拥有系统的源代码，我们可以通过添加把DKK转换到USD的新函数来扩展系统。但是，如果我们没有访问应用的源代码权限，因为这个系统仅支持扩展库，该怎么办呢？遇到这种情况，我们仍旧使用库（例如，调用系统的方法），但是不能修改／扩展它。解决办法是编写一个包装器（也被称作适配器）把数据从给定的DKK格式转换到希望使用的USD或者EUR格式。

可能我们整天都在使用适配器模式，不过用的是硬件而不是软件。如果你拥有一部智能手机或者一台平板，那么你需要使用为了把设备连接到电脑而实用一个USB适配器。如果从欧洲的大多数国家启程去到英国旅行的话，你需要为你的笔记本电脑使用一个电源适配器。如果你从欧洲到美国旅行，或者周边国家时，也是同样的结果。适配器无处不在！

适配器模式在实现之后用来让事务工作起来。通常两个不兼容的接口中的其中一个不是外部的，就是老旧的。如果接口是外部的，这意味着我们不能访问源代码。如果是旧的话，一般不实际的做法是重构它。我们进一步来探讨改变旧代码组建的实现来实现我们的需求不仅仅是不切实际的，而且这还违反了开发／关闭原则。开发／关闭原则是面向对象设计的一个基础原则。它声明了，软件对象应该为扩展而开发，且对修改进行关闭。这基本就是说，我们应该能够扩展一个对象的行为，而不用对源代码进行修改。适配器同样遵守开发／关闭原则。

### 实现     
在Python中有很多实现适配器模式的方法。所有的技术都由布鲁斯－埃寇使用继承来阐明，但是Python提供了一个选择，在我看来是一种更为理想的实现适配器的办法。另外一种技术你应该是熟悉的，因此它使用的是类的内部字典，我们已经见过如何具体操作了。


In [ ]:
现在，进入我们我们想要使用的代码。我们决定使用更多的功能扩展我们的应用，幸运的是，我们找到了两个有趣的使用不同和我们的应用无关的库实现的类：Musician 和 Dancer。在Musician，主要的行为是由play()方法执行的。在Dancer类中，主要是由dance()执行的。为了表明这两个类是来自外部的，我们把它放到一个独立的模块中，一如下所示：

In [3]:
class Musician: 
    def __init__(self, name): 
        self.name = name
 
    def __str__(self): 
        return 'the musician {}'.format(self.name)
 
    def play(self): 
        return 'plays music' 
    
class Dancer: 
    def __init__(self, name): 
        self.name = name 
 
    def __str__(self): 
        return 'the dancer {}'.format(self.name)
 
    def dance(self): 
        return 'does a dance performance'

In [4]:
class Club: 
    def __init__(self, name): 
        self.name = name 
 
    def __str__(self): 
        return 'the club {}'.format(self.name)
 
    def organize_event(self): 
        return 'hires an artist to perform for the people' 

到目前为止还好。但是，我们有一个问题。用户仅知道如何调用organize_event()方法，而不知道如何使用play()或者dance()。我们如何让代码工作而又不改变类miusic和dance？适配器就是拯救这个问题的！我们创建一个允许我们让不同的接口的多个对象适配到一个统一的接口的通用适配器类。__init__()的obj参数是我们想要去适配的对象，而adapted_methods则是一个字典。

In [5]:
class Adapter: 
    def __init__(self, obj, adapted_methods): 
        self.obj = obj 
        self.__dict__.update(adapted_methods) 
 
    def __str__(self): 
        return str(self.obj) 

让我们来看看如何使用适配器模式。一个对象列表拥有多个对象。属于Computer类的合适对象不需要去适配。我们直接把它们加入到列表。不合适的对象不会直接添加。它们要使用Adapter类进行适配。其结果是客户端代码可以继续在所有对象上使用 organize_event() 方法，而不需要关心所使用类之间任何接口上的不同。

In [6]:
def main(): 

    objects = [Club('Jazz Cafe'), Musician('Roy Ayers'), Dancer('Shane Sparks')]
    
    for obj in objects:
        if hasattr(obj, 'play') or hasattr(obj, 'dance'):
            if hasattr(obj, 'play'):
                adapted_methods = dict(organize_event=obj.play)
            elif hasattr(obj, 'dance'):            
                adapted_methods = dict(organize_event=obj.dance)
                
            # referencing the adapted object here
            obj = Adapter(obj, adapted_methods)
            
        print('{} {}'.format(obj,obj.organize_event())) 

In [7]:
main()

the club Jazz Cafe hires an artist to perform for the people
the musician Roy Ayers plays music
the dancer Shane Sparks does a dance performance
